# Tutorial 03: Running RLlib Experiments

This tutorial walks you through the process of running traffic simulations in Flow with trainable RLlib-powered agents. Autonomous agents will learn to maximize a certain reward over the rollouts, using the [**RLlib**](https://ray.readthedocs.io/en/latest/rllib.html) library ([citation](https://arxiv.org/abs/1712.09381)) ([installation instructions](https://flow.readthedocs.io/en/latest/flow_setup.html#optional-install-ray-rllib)). Simulations of this form will depict the propensity of RL agents to influence the traffic of a human fleet in order to make the whole fleet more efficient (for some given metrics). 

本教程将带您完成使用可训练的rllib支持的代理在流中运行交通模拟的过程。自治代理将学习一定的回报最大化的卷输出,使用[* * RLlib * *] (https://ray.readthedocs.io/en/latest/rllib.html)图书馆([引用](https://arxiv.org/abs/1712.09381))((安装说明)(https://flow.readthedocs.io/en/latest/flow_setup.html optional-install-ray-rllib))。这种形式的模拟将描述RL代理影响人类舰队的流量的倾向，以使整个舰队更有效率(对于某些给定的度量)。

In this tutorial, we simulate an initially perturbed single lane ring road, where we introduce a single autonomous vehicle. We witness that, after some training, that the autonomous vehicle learns to dissipate the formation and propagation of "phantom jams" which form when only human driver dynamics are involved.

在本教程中，我们模拟了一个最初受到干扰的单车道环路，其中我们引入了一辆自动驾驶汽车。我们可以看到，经过一些训练后，自动驾驶汽车学会了消除“幻影交通堵塞”的形成和传播，而这种“幻影交通堵塞”是在只有人类驾驶员的动态参与的情况下形成的。

## 1. Components of a Simulation
All simulations, both in the presence and absence of RL, require two components: a *network*, and an *environment*. Networks describe the features of the transportation network used in simulation. This includes the positions and properties of nodes and edges constituting the lanes and junctions, as well as properties of the vehicles, traffic lights, inflows, etc... in the network. Environments, on the other hand, initialize, reset, and advance simulations, and act as the primary interface between the reinforcement learning algorithm and the network. Moreover, custom environments may be used to modify the dynamical features of an network. Finally, in the RL case, it is in the *environment* that the state/action spaces and the reward function are defined. 

所有的模拟，无论是否存在RL，都需要两个组件:一个“网络”和一个“环境”。网络描述了用于模拟的交通网络的特征。这包括构成车道和路口的节点和边缘的位置和属性，以及车辆、红绿灯、流入量等的属性。在网络。环境，另一方面，初始化，重置，并推进模拟，并作为加强学习算法和网络之间的主要接口。此外，可以使用自定义环境来修改网络的动态特性。最后，在RL案例中，状态/动作空间和奖励函数是在*environment*中定义的。

## 2. Setting up a Network
Flow contains a plethora of pre-designed networks used to replicate highways, intersections, and merges in both closed and open settings. All these networks are located in flow/networks. For this tutorial, which involves a single lane ring road, we will use the network `RingNetwork`.

Flow包含大量预先设计的网络，用于在封闭和开放环境中复制高速公路、十字路口和合并。所有这些网络都位于流/网络中。本教程涉及单行道，我们将使用网络`RingNetwork`.

### 2.1 Setting up Network Parameters

The network mentioned at the start of this section, as well as all other networks in Flow, are parameterized by the following arguments: 
本节开头提到的网络，以及Flow中的所有其他网络，都是由以下参数参数化的:
* name
* vehicles
* net_params
* initial_config

These parameters are explained in detail in `tutorial01_sumo.ipynb`. Moreover, all parameters excluding vehicles (covered in section 2.2) do not change from the previous tutorial. Accordingly, we specify them nearly as we have before, and leave further explanations of the parameters to `tutorial01_sumo.ipynb`.

这些参数在“tutorial01_sumo.ipynb”中详细解释。此外，除车辆外的所有参数(第2.2节中涉及的参数)与前一教程没有变化。因此，我们几乎像以前那样指定它们，并将参数的进一步解释留给' tutorial01_sumo.ipynb '。

We begin by choosing the network the experiment will be trained on. We use one of Flow's builtin networks, located in `flow.networks`. A list of all available networks can be found by running the script below.

我们首先选择实验要训练的网络。我们使用Flow的一个内置网络，位于“Flow .networks”中。运行下面的脚本可以找到所有可用网络的列表。

In [ ]:
import flow.networks as networks

print(networks.__all__)

In this tutorial, we choose to use the ring road network. The network class is then:
在本教程中，我们选择使用环路网络。网络类则为:

In [ ]:
from flow.networks import RingNetwork

# ring road network class
network_name = RingNetwork

One key difference between SUMO and RLlib experiments is that, in RLlib experiments, the network classes do not need to be defined; instead users should simply name the network class they wish to use. Later on, an environment setup module will import the correct network class based on the provided names.

SUMO和RLlib实验的一个关键区别是，在RLlib实验中，不需要定义网络类;相反，用户应该简单地命名他们希望使用的网络类。稍后，环境设置模块将根据提供的名称导入正确的网络类。

In [ ]:
# input parameter classes to the network class
from flow.core.params import NetParams, InitialConfig

# name of the network
name = "training_example"

# network-specific parameters
from flow.networks.ring import ADDITIONAL_NET_PARAMS
net_params = NetParams(additional_params=ADDITIONAL_NET_PARAMS)

# initial configuration to vehicles
initial_config = InitialConfig(spacing="uniform", perturbation=1)

### 2.2 Adding Trainable Autonomous Vehicles增加可训练的自动驾驶车辆
The `Vehicles` class stores state information on all vehicles in the network. This class is used to identify the dynamical features of a vehicle and whether it is controlled by a reinforcement learning agent. Morover, information pertaining to the observations and reward function can be collected from various `get` methods within this class.

“车辆”类存储网络中所有车辆的状态信息。这个类别是用来识别车辆的动力特性，以及它是否由强化学习代理控制。Morover，与观察和奖励函数相关的信息可以从这个类中的各种' get '方法中收集。

The dynamics of vehicles in the `Vehicles` class can either be depicted by sumo or by the dynamical methods located in flow/controllers. For human-driven vehicles, we use the IDM model for acceleration behavior, with exogenous gaussian acceleration noise with std 0.2 m/s2 to induce perturbations that produce stop-and-go behavior. In addition, we use the `ContinousRouter` routing controller so that the vehicles may maintain their routes closed networks.

“车辆”类中的车辆动力学可以用sumo来描述，也可以用流/控制器中的动力学方法来描述。对于人驾驶车辆，我们使用IDM模型进行加速行为，使用std 0.2 m/s2的外源性高斯加速度噪声来诱导扰动，从而产生走走停停的行为。此外，我们使用“ContinousRouter”的路由控制器，使车辆可以保持其路线的封闭网络。

As we have done in `tutorial01_sumo.ipynb`, human-driven vehicles are defined in the `VehicleParams` class as follows:

就像我们在“tutorial01_sumo”中所做的那样。人驾驶车辆在“车辆参数”类别中定义如下:

In [ ]:
# vehicles class
from flow.core.params import VehicleParams

# vehicles dynamics models
from flow.controllers import IDMController, ContinuousRouter

vehicles = VehicleParams()
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=21)

The above addition to the `Vehicles` class only accounts for 21 of the 22 vehicles that are placed in the network. We now add an additional trainable autuonomous vehicle whose actions are dictated by an RL agent. This is done by specifying an `RLController` as the acceleraton controller to the vehicle. 

上述添加到“车辆”类的车辆仅占网络中放置的22辆车辆中的21辆。我们现在添加了一个额外的可训练的自动驾驶车辆，它的行动是由一个RL代理决定的。这是通过指定一个“RLController”作为车辆的加速控制器来实现的。

In [ ]:
from flow.controllers import RLController

Note that this controller serves primarirly as a placeholder that marks the vehicle as a component of the RL agent, meaning that lane changing and routing actions can also be specified by the RL agent to this vehicle.

We finally add the vehicle as follows, while again using the `ContinuousRouter` to perpetually maintain the vehicle within the network.

注意，这个控制器主要用作占位符，它将车辆标记为RL代理的一个组件，这意味着RL代理也可以为该车辆指定变道和路由操作。

我们最终如下所示添加了车辆，同时再次使用“ContinuousRouter”来永久维护网络中的车辆。

In [ ]:
vehicles.add(veh_id="rl",
             acceleration_controller=(RLController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=1)

## 3. Setting up an Environment

Several environments in Flow exist to train RL agents of different forms (e.g. autonomous vehicles, traffic lights) to perform a variety of different tasks. The use of an environment allows us to view the cumulative reward simulation rollouts receive, along with to specify the state/action spaces.

在流中存在一些环境来训练不同形式的RL代理(例如，自动车辆、交通灯)执行各种不同的任务。环境的使用允许我们查看累积的奖励模拟结果，以及指定状态/动作空间。

Sumo envrionments in Flow are parametrized by three components:
流中的sumo环境由三个部分参数化:
* `SumoParams`
* `EnvParams`
* `Network`

### 3.1 SumoParams
`SumoParams` specifies simulation-specific variables. These variables include the length of any simulation step and whether to render the GUI when running the experiment. For this example, we consider a simulation step length of 0.1s and deactivate the GUI. 

“SumoParams”指定特定于模拟的变量。这些变量包括任何模拟步骤的长度以及在运行实验时是否呈现GUI。对于本例，我们考虑仿真步骤长度为0.1s并停用GUI。

**Note** For training purposes, it is highly recommanded to deactivate the GUI in order to avoid global slow down. In such case, one just needs to specify the following: `render=False`

**注**出于培训目的，强烈建议禁用GUI以避免全局变慢。在这种情况下，只需指定以下内容:' render=False '

In [ ]:
from flow.core.params import SumoParams

sim_params = SumoParams(sim_step=0.1, render=False)

### 3.2 EnvParams

`EnvParams` specifies environment and experiment-specific parameters that either affect the training process or the dynamics of various components within the network. For the environment `WaveAttenuationPOEnv`, these parameters are used to dictate bounds on the accelerations of the autonomous vehicles, as well as the range of ring lengths (and accordingly network densities) the agent is trained on.

“EnvParams”指定了环境和实验特定的参数，这些参数要么影响训练过程，要么影响网络中各种组件的动态。对于环境“WaveAttenuationPOEnv”，这些参数用于规定自动驾驶车辆的加速界限，以及agent所训练的环长(以及相应的网络密度)范围。

Finally, it is important to specify here the *horizon* of the experiment, which is the duration of one episode (during which the RL-agent acquire data). 

最后，在这里指定实验的“horizon”是很重要的，它是一个阶段的持续时间(在此期间，RL-agent获取数据)。

In [ ]:
from flow.core.params import EnvParams

# Define horizon as a variable to ensure consistent use across notebook
HORIZON=100

env_params = EnvParams(
    # length of one rollout
    horizon=HORIZON,

    additional_params={
        # maximum acceleration of autonomous vehicles
        "max_accel": 1,
        # maximum deceleration of autonomous vehicles
        "max_decel": 1,
        # bounds on the ranges of ring road lengths the autonomous vehicle 
        # is trained on
        "ring_length": [220, 270],
    },
)

### 3.3 Initializing a Gym Environment 初始化一个训练环境

Now, we have to specify our Gym Environment and the algorithm that our RL agents will use. Similar to the network, we choose to use on of Flow's builtin environments, a list of which is provided by the script below.

现在，我们必须指定健身房环境和RL代理将使用的算法。与网络类似，我们选择在Flow的构建环境上使用，下面的脚本提供了一个列表。

In [ ]:
import flow.envs as flowenvs

print(flowenvs.__all__)

We will use the environment "WaveAttenuationPOEnv", which is used to train autonomous vehicles to attenuate the formation and propagation of waves in a partially observable variable density ring road. To create the Gym Environment, the only necessary parameters are the environment name plus the previously defined variables. These are defined as follows:

我们将使用环境“WaveAttenuationPOEnv”，该环境用于训练自动驾驶车辆在部分可观测变密度环路中衰减波的形成和传播。要创建健身房环境，惟一需要的参数是环境名称和前面定义的变量。这些定义如下:

In [ ]:
from flow.envs import WaveAttenuationPOEnv

env_name = WaveAttenuationPOEnv

### 3.4 Setting up Flow Parameters 设置流量参数

RLlib experiments both generate a `params.json` file for each experiment run. For RLlib experiments, the parameters defining the Flow network and environment must be stored as well. As such, in this section we define the dictionary `flow_params`, which contains the variables required by the utility function `make_create_env`. `make_create_env` is a higher-order function which returns a function `create_env` that initializes a Gym environment corresponding to the Flow network specified.

RLlib实验都生成一个“参数”。每个实验运行的json文件。对于RLlib实验，定义流网络和环境的参数也必须存储。因此，在本节中，我们定义字典“flow_params”，它包含实用函数“make_create_env”所需的变量。“make_create_env”是一个高阶函数，它返回一个函数“create_env”，该函数初始化一个健身房环境，该环境与指定的流网络相对应。

In [ ]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict(
    # name of the experiment
    exp_tag=name,
    # name of the flow environment the experiment is running on
    env_name=env_name,
    # name of the network class the experiment uses
    network=network_name,
    # simulator that is used by the experiment
    simulator='traci',
    # simulation-related parameters
    sim=sim_params,
    # environment related parameters (see flow.core.params.EnvParams)
    env=env_params,
    # network-related parameters (see flow.core.params.NetParams and
    # the network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,
    # vehicles to be placed in the network at the start of a rollout 
    # (see flow.core.vehicles.Vehicles)
    veh=vehicles,
    # (optional) parameters affecting the positioning of vehicles upon 
    # initialization/reset (see flow.core.params.InitialConfig)
    initial=initial_config
)

## 4 Running RL experiments in Ray 在Ray中运行RL实验

### 4.1 Import 

First, we must import modules required to run experiments in Ray. The `json` package is required to store the Flow experiment parameters in the `params.json` file, as is `FlowParamsEncoder`. Ray-related imports are required: the PPO algorithm agent, `ray.tune`'s experiment runner, and environment helper methods `register_env` and `make_create_env`.

首先，我们必须导入在Ray中运行实验所需的模块。“json”包需要将流实验参数存储在“params.json '文件，如' FlowParamsEncoder '。需要与ray相关的导入:PPO算法代理“ray”。调优的实验运行器和环境助手方法' register_env '和' make_create_env '。

In [ ]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

### 4.2 Initializing Ray 初始化ray
Here, we initialize Ray and experiment-based constant variables specifying parallelism in the experiment as well as experiment batch size in terms of number of rollouts.

在这里，我们初始化了Ray和基于实验的常量变量，这些变量指定了实验中的并行性（cpu），以及实验批量大小。

In [ ]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 1

ray.init(num_cpus=N_CPUS)

### 4.3 Configuration and Setup 配置和设置
Here, we copy and modify the default configuration for the [PPO algorithm](https://arxiv.org/abs/1707.06347). The agent has the number of parallel workers specified, a batch size corresponding to `N_ROLLOUTS` rollouts (each of which has length `HORIZON` steps), a discount rate $\gamma$ of 0.999, two hidden layers of size 16, uses Generalized Advantage Estimation, $\lambda$ of 0.97, and other parameters as set below.

在这里，我们复制并修改了[PPO算法](https://arxiv.org/abs/1707.06347)的默认配置。代理指定了并行工作者的数量，批大小对应于' N_ROLLOUTS ' rollouts(每个都有长度' HORIZON '步骤)，折现率$\gamma$为0.999，两个大小为16的隐藏层，使用广义优势估计，$\lambda$为0.97，以及其他参数，如下所示。

Once `config` contains the desired parameters, a JSON string corresponding to the `flow_params` specified in section 3 is generated. The `FlowParamsEncoder` maps objects to string representations so that the experiment can be reproduced later. That string representation is stored within the `env_config` section of the `config` dictionary. Later, `config` is written out to the file `params.json`. 

一旦“config”包含所需的参数，就会生成与第3节中指定的“flow_params”对应的JSON字符串。“FlowParamsEncoder”将对象映射到字符串表示，以便以后可以复制实验。该字符串表示形式存储在' config '字典的' env_config '节中。稍后，' config '被写入文件' params.json '。

Next, we call `make_create_env` and pass in the `flow_params` to return a function we can use to register our Flow environment with Gym. 

接下来，我们调用“make_create_env”并传入“flow_params”来返回一个函数，我们可以使用该函数向Gym注册流环境。

In [ ]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS - 1  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [16, 16]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

### 4.4 Running Experiments 运行实验

Here, we use the `run_experiments` function from `ray.tune`. The function takes a dictionary with one key, a name corresponding to the experiment, and one value, itself a dictionary containing parameters for training.

这里，我们使用了' ray.tune '中的' run_experiments '函数。该函数使用一个键、一个与实验对应的名称和一个值组成的字典，字典本身包含用于训练的参数。

In [ ]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "checkpoint_at_end": True,  # generate a checkpoint at the end
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1,  # number of iterations to stop after
        },
    },
})

### 4.5 Visualizing the results 可视化结果

The simulation results are saved within the `ray_results/training_example` directory (we defined `training_example` at the start of this tutorial). The `ray_results` folder is by default located at your root `~/ray_results`. 

模拟结果保存在“ray_results/training_example”目录中(我们在本教程开始时定义了“training_example”)。默认情况下，' ray_results '文件夹位于根目录' ~/ray_results '。

You can run `tensorboard --logdir=~/ray_results/training_example` (install it with `pip install tensorboard`) to visualize the different data outputted by your simulation.

您可以运行“tensorboard—logdir=~/ray_results/training_example”(将其与“pip install tensorboard”一起安装)来可视化仿真输出的不同数据。

For more instructions about visualizing, please see `tutorial05_visualize.ipynb`. 

有关可视化的更多说明，请参见“tutorial05_visualize.ipynb”。

### 4.6 Restart from a checkpoint / Transfer learning 从一个检查点/转移学习重新开始

If you wish to do transfer learning, or to resume a previous training, you will need to start the simulation from a previous checkpoint. To do that, you can add a `restore` parameter in the `run_experiments` argument, as follows:

如果您希望进行转移学习，或恢复以前的培训，您将需要从以前的检查点开始模拟。为此，您可以在“run_tests”参数中添加一个“restore”参数，如下所示:

```python
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "restore": "/ray_results/experiment/dir/checkpoint_50/checkpoint-50"
        "checkpoint_freq": 1,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 1,
        },
    },
})
```

The `"restore"` path should be such that the `[restore]/.tune_metadata` file exists.
“restore”路径应该是“[restore]/.tune_metadata”文件存在。

There is also a `"resume"` parameter that you can set to `True` if you just wish to continue the training from a previously saved checkpoint, in case you are still training on the same experiment. 

还有一个“resume”参数，如果您只是想从以前保存的检查点继续训练，那么您可以将其设置为“True”，以防您仍然在同一个实验上进行训练。